In [63]:
import pandas as pd
import numpy as np

#steve-report is a recording of my shifts to-date. routes.csv is all routes from Google GTFS
driver_report = pd.read_csv('driver-report.csv')
routes = pd.read_csv('busroutes.csv')
#print(routes)
driver_report

FileNotFoundError: [Errno 2] No such file or directory: 'driver-report.csv'

In [58]:
#These steps don't seem to work:

#remove decimal and trailing zero from data
driver_report['route'] = driver_report.route.astype(str)
driver_report['route'] = driver_report.route.str.replace('.0', ' ')
#Drop NaN values (Service, Training, etc)
driver_report['route'] = driver_report.route.dropna()

#routes["route_id"] = routes.route_id.astype(str)

#change the report column back to numbers so it can be compared to the other CSV:
driver_report["route"] = pd.to_numeric(driver_report["route"], errors='ignore')
#same with the Routes .csv:
routes["route_id"] = pd.to_numeric(routes["route_id"], errors='ignore')



/var/folders/gt/yj438g6s03dchf7qmhfj45640000gn/T/ipykernel_781/3454818182.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  driver_report['route'] = driver_report.route.str.replace('.0', ' ')


In [59]:
#these work for exact match but not fuzzy match
print(routes[routes.route_short_name.isin(driver_report.route)])
#print(driver_report[driver_report.route.isin(routes.route_id)])

#driver_report
#routes


Empty DataFrame
Columns: [route_id, route_short_name, route_long_name, route_desc, route_type, route_url]
Index: []


In [60]:
#Almost giving up. Let's try to join them?

df = pd.merge(driver_report, routes, on=['route', 'route_id'])

KeyError: 'route'

In [6]:

#pre-2014 legacy bus shift naming convention not captured here.

am_shifts = '^[0-9]{2}-[0-9] AM(.*)$'
mid_shifts = '^[0-9]{2}-[0-9] MID(.*)$'
pm_shifts = '^[0-9]{2}-[0-9] PM(.*)$'
pm2_shifts = '^[0-9]{2}-[0-9] PM2(.*)$'
eve_shifts = '^[0-9]{2}-[0-9] EVE(.*)$'

#this gives me a rudimentary way of seeing what shifts I did in morning, afternoon, and evening blocks. For example:
#print(driver_report[driver_report.shiftname.str.contains(mid_shifts)])

#driver_report[driver_report.shiftname.str.contains(mid_shifts)]

#I can join these regexes together and see all shifts:
allshifts = [am_shifts, mid_shifts, pm_shifts, pm2_shifts, eve_shifts]
pattern = ('|'.join(allshifts))
print(driver_report[driver_report.shiftname.str.contains(pattern)])

#(I can also just sort on "bus")



            shiftname start time end time  length        date route department
220           30-1 AM       6:55    11:00    4.00  2014-08-01              Bus
221          32-1 MID      11:25    14:40    3.25  2014-08-01   32         Bus
222           32-1 AM       7:00    10:18    3.25  2014-08-05   32         Bus
223           31-3 AM       6:21    11:16    5.00  2014-08-06   31         Bus
224           30-1 AM       6:55    11:00    4.00  2014-08-07              Bus
...               ...        ...      ...     ...         ...   ...        ...
1362  31-3 PM1 Part 1      13:51    15:00    1.25  2018-05-08   31         Bus
1363  31-3 PM1 Part 2      15:00    15:06    0.00  2018-05-08   31         Bus
1364   33-2 AM Part 1       7:21     8:51    1.50  2018-05-10   33         Bus
1366   36-1 AM Part 2       8:47    10:15    1.50  2018-06-08   36         Bus
1367   36-1 AM Part 3      10:15    10:17    0.00  2018-06-08   36         Bus

[545 rows x 7 columns]


/var/folders/gt/yj438g6s03dchf7qmhfj45640000gn/T/ipykernel_781/3270977363.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print(driver_report[driver_report.shiftname.str.contains(pattern)])


In [17]:
#Now I want to analyze the data.

#average length of a MID shift
driver_report.length[driver_report.shiftname.str.contains(mid_shifts)].describe()

/var/folders/gt/yj438g6s03dchf7qmhfj45640000gn/T/ipykernel_781/4036545774.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  driver_report.length[driver_report.shiftname.str.contains(mid_shifts)].describe()


count    167.000000
mean       2.732036
std        1.006967
min        0.250000
25%        2.500000
50%        3.000000
75%        3.250000
max        5.000000
Name: length, dtype: float64

In [16]:
import folium

folium_map = folium.Map(location=[42.375968582346225, -72.520124958991],
                    zoom_start = 13) 
folium_map # Calls the map to display


In [ ]:
#heatmap

from folium import plugins
from folium.plugins import HeatMap

df_acc['Latitude'] = df_acc['Latitude'].astype(float)
df_acc['Longitude'] = df_acc['Longitude'].astype(float)

# Ensure you're handing it floats
df_acc['Latitude'] = df_acc['Latitude'].astype(float)
df_acc['Longitude'] = df_acc['Longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df = df_acc[df_acc['Speed_limit']=='40'] # Reducing data size so it runs faster
heat_df = df_acc[df_acc['Year']=='2007'] # Reducing data size so it runs faster
heat_df = heat_df[['Latitude', 'Longitude']]
heat_df = heat_df.dropna(axis=0, subset=['Latitude','Longitude'])

# List comprehension to make out list of lists
heat_data = [[row['Latitude'],row['Longitude']] for index, row in heat_df.iterrows()]

# Plot it on the map
HeatMap(heat_data).add_to(map_hooray)

# Display the map
map_hooray
